In [ ]:
import os
import rasterio as rio
import numpy as np
import pandas as pd
from copy import deepcopy
import random
from joblib import Parallel, delayed
from tqdm import tqdm

In [ ]:
maindir1 = r'G:\CopernicusTurbidityLayers'
subdir1 = os.path.join(maindir1,'masked')
maindir2 = r'G:\ReflectanceLayers'
cases = ['withoutGF','withGF']
for case in cases:
    subdir2 = os.path.join(maindir2,case,'France')
    maindir3= r'G:\preparedInputData'
    dates = os.listdir(subdir2)
    tenDays = np.array([[0,0,0,0]])
    for i in range(0,len(dates),4):                         
        tenDays = np.r_[tenDays,[[dates[i+k] for k in range(4)]]] 
    tenDays = tenDays[1:]
    for k in range(15):
        file1 = os.listdir(subdir1)[k]
        turImg = rio.open(os.path.join(subdir1,file1))
        turArr = turImg.read()
        myarray = turArr.copy()                          
        for date in tenDays[k]:                              
            subdir3 = os.path.join(subdir2,date)        
            for layer in os.listdir(subdir3):                           
                reflectanceArr = rio.open(os.path.join(subdir3,layer)).read()
                myarray = np.append(myarray, reflectanceArr, axis=0)
        outputdir = os.path.join(maindir3,case,'France', 'merged_'+file1)
        with rio.open(outputdir,'w',driver='Gtiff', width=turImg.width, height=turImg.height, 
                            count=37,crs=turImg.crs,transform=turImg.transform, dtype='float32', nodata=9.96921e+36) as newImg:
            for k in range(len(myarray)):
                newImg.write(myarray[k,:,:], indexes=k+1)
            newImg.close()

In [ ]:
maindir1 = r'G:\preparedInputData'
subdir1 = os.path.join(maindir1,'withoutGF')
nb_none = 0
nb_UP = 0
nb_KP = 0
for file1 in os.listdir(subdir1):
    if file1.endswith('tiff'):
        img = rio.open(os.path.join(subdir1,file1)) 
        arr = img.read()        
        turLayer = arr[0]
        for i in range(turLayer.shape[0]):
            for j in range(turLayer.shape[1]):
                if turLayer[i,j]>1e+36:
                    nb_none+=1
                elif turLayer[i,j]<0:
                    nb_UP+=1
                else:
                    nb_KP+=1        
        print(file1[7:15],'nb_none:',nb_none,'__','nb_UP:',nb_UP,'__','5% nb_KP:',int(nb_KP*0.05))

In [ ]:
def getPixelValue(array,idx1,idx2,idx3):
    return array[idx1,idx2,idx3]

In [ ]:
### without GF ### clear sky ###

maindir1 = r'G:\MScThesis\waterQualityMonitoring\Data\MLearning\preparedInputData'
subdir1 = os.path.join(maindir1,'withoutGF','France')
for file1 in os.listdir(subdir1):
    if file1.endswith('tiff'):
        img1 = rio.open(os.path.join(subdir1,file1)) # start by reading all layers
        arr1 = img1.read()
        for idx in range(arr1.shape[1]):
            for idy in range(arr1.shape[2]):
                if arr1[0,idx,idy] >= 1e+36:
                    for n in range(1,arr1.shape[0]):
                        arr1[n,idx,idy] = 9.96921e+36
        indices = [i for i in range(1,36,9)]
        nb_KP = []
        for i in indices:
            temp_copy = deepcopy(arr1[i])
            temp_copy[temp_copy==-99]=9.96921e+36
            nb_KP.append(len(np.argwhere(temp_copy<=1e+36).tolist()))        
        df1 = pd.DataFrame({'indices':indices,'nb_KP':nb_KP})
        df1.sort_values('nb_KP', inplace=True)  
        df1.reset_index(drop=True, inplace=True) 
        for i in tqdm([0,1,2,3]): 
            name = 'Pixels_From_Best_'+str(4-i)+'_S2_'+file1[7:15]           
            l = list(df1[i:]['indices'])   
            arr1_temp = np.expand_dims(arr1[0], axis=0)
            for k1 in l: 
                for k2 in range(k1,k1+9):
                    arr1_temp = np.append(arr1_temp,np.expand_dims(arr1[k2], axis=0),axis=0)
            list_coordKP = np.argwhere(arr1_temp[0,:,:]<1e+36).tolist()
            list_KP_Values = list(arr1_temp[0,:,:][arr1_temp[0,:,:]<1e+36])
            m = int(np.min(arr1_temp[0,:,:][arr1_temp[0,:,:]<1e+36]))
            M = int(np.max(arr1_temp[0,:,:][arr1_temp[0,:,:]<1e+36]))
            list_boundary_values = []
            for t in range(m,M,20):            
                list_boundary_values.append(t)
            if M not in list_boundary_values:
                list_boundary_values.append(M+1)
            list_names=['list'+str(i) for i in range(len(list_boundary_values)-1)]
            groups_KP1 = {}
            for list_name in list_names:
                groups_KP1[list_name] = []
            for _ in range(len(list_KP_Values)):              
                for i in range(len(list_boundary_values)-1): 
                    if (list_KP_Values[_]>=list_boundary_values[i]) and (list_KP_Values[_]<list_boundary_values[i+1]):
                        groups_KP1['list'+str(i)].append(list_coordKP[_])
            n_KP_noGF = 0
            for key in groups_KP1.keys():               
                idX1_temp = []
                idY1_temp = []
                for i1 in range(len(groups_KP1[key])):
                    x_temp, y_temp = groups_KP1[key][i1]
                    vector = [arr1_temp[m, x_temp, y_temp] for m in range(len(arr1_temp))]
                    missingData = vector.count(-99)   
                    if missingData == 0:    
                        idX1_temp.append(x_temp)
                        idY1_temp.append(y_temp)                    
                groups_KP1[key] = list(zip(idX1_temp, idY1_temp)) 
                n_KP_noGF+=len(list(zip(idX1_temp, idY1_temp)))       
            if (0.05*n_KP_noGF) > 5001:
                n1 = 0.05*n_KP_noGF  
            else:
                if n_KP_noGF > 5001:
                    n1 = 5001
                else:
                    n1 = n_KP_noGF                    
            idX1 = []
            idY1 = []
            for key in groups_KP1.keys():                
                random.seed(4)
                list_temp1 = random.sample(groups_KP1[key],len(groups_KP1[key])) 
                if n_KP_noGF > 0:
                    percent = len(groups_KP1[key])/n_KP_noGF                            
                    n2 = int(percent*n1)              
                    if n2 > 0:
                        for i1 in range(n2):
                            x_temp1, y_temp1 = list_temp1[i1]  
                            idX1.append(x_temp1)
                            idY1.append(y_temp1)          
            rows1 = ['L'+str(index) for index in range(len(arr1_temp))]
            columns1 = [index for index in range(len(idX1))]
            results1 = pd.DataFrame(index=rows1, columns=columns1)
            data1 = [] 
            for idxLayer in range(len(arr1_temp)):
                pixelValues = Parallel(n_jobs=-1)(delayed(getPixelValue)(arr1_temp,idxLayer,idX1[k],idY1[k]) for k in range(len(idX1)))
                data1.append(pixelValues)
            results1 = pd.DataFrame(data1, index=rows1, columns=columns1).T
            results1.insert(loc=0, column='idx', value=idX1)   
            results1.insert(loc=1, column='idy', value=idY1)
            path1 = os.path.join(subdir1,file1[7:15])
            os.makedirs(path1, exist_ok=True)
            outputdir1 = os.path.join(path1, name+'.xlsx')
            results1.to_excel(outputdir1, encoding='utf-8') 

In [ ]:
### with GF ### different amounts of gaps ###

maindir1 = r'G:\MScThesis\waterQualityMonitoring\Data\MLearning\preparedInputData'
subdir0 = os.path.join(maindir1,'withoutGF','France')  
subdir1 = os.path.join(maindir1,'withGF','France')
for file1 in os.listdir(subdir1):
    if file1.endswith('tiff'):
        arr0 = rio.open(os.path.join(subdir0,file1)).read()
        img1 = rio.open(os.path.join(subdir1,file1)) 
        arr1 = img1.read()
        for idx in range(arr0.shape[1]):
            for idy in range(arr0.shape[2]):
                if arr0[0,idx,idy] >= 1e+36:
                    for n in range(1,arr0.shape[0]):
                        arr0[n,idx,idy] = 9.96921e+36
        indices = [i for i in range(1,36,9)]
        nb_KP = []
        for i in indices:
            temp_copy = deepcopy(arr0[i])
            temp_copy[temp_copy==-99]=9.96921e+36
            nb_KP.append(len(np.argwhere(temp_copy<=1e+36).tolist()))        
        df1 = pd.DataFrame({'indices':indices,'nb_KP':nb_KP})
        df1.sort_values('nb_KP', inplace=True)  
        df1.reset_index(drop=True, inplace=True) 
        for i in tqdm([1,2,3]):
            name = 'Pixels_From_Best_'+str(4-i)+'_S2_'+file1[7:15]           
            l = list(df1[i:]['indices'])   
            arr1_temp = np.expand_dims(arr1[0], axis=0)            
            for k1 in l:
                for k2 in range(k1,k1+9):
                    arr1_temp = np.append(arr1_temp,np.expand_dims(arr1[k2], axis=0),axis=0) 
            list_coordKP = np.argwhere(arr1_temp[0,:,:]<1e+36).tolist()
            list_KP_Values = list(arr1_temp[0,:,:][arr1_temp[0,:,:]<1e+36])
            m = int(np.min(arr1_temp[0,:,:][arr1_temp[0,:,:]<1e+36]))
            M = int(np.max(arr1_temp[0,:,:][arr1_temp[0,:,:]<1e+36]))
            list_boundary_values = []
            for t in range(m,M,20):            
                list_boundary_values.append(t)
            if M not in list_boundary_values:
                list_boundary_values.append(M+1)
            list_names=['list'+str(i) for i in range(len(list_boundary_values)-1)]
            groups_KP1 = {}
            for list_name in list_names:
                groups_KP1[list_name] = []
            for _ in range(len(list_KP_Values)):              
                for i in range(len(list_boundary_values)-1): 
                    if (list_KP_Values[_]>=list_boundary_values[i]) and (list_KP_Values[_]<list_boundary_values[i+1]):
                        groups_KP1['list'+str(i)].append(list_coordKP[_]) 
            n_KP_GF = 0 
            for key in groups_KP1.keys():               
                idX1_temp = []
                idY1_temp = []
                for i1 in range(len(groups_KP1[key])):
                    x_temp, y_temp = groups_KP1[key][i1]
                    vector = [arr1_temp[m, x_temp, y_temp] for m in range(len(arr1_temp))] 
                    missingData = vector.count(-99)                       
                    if i==1: # best3
                        MD = 19
                    elif i==2 or i==3: 
                        MD = 10     
                    if missingData < MD:    
                        idX1_temp.append(x_temp)
                        idY1_temp.append(y_temp)                 
                groups_KP1[key] = list(zip(idX1_temp, idY1_temp)) 
                n_KP_GF+=len(list(zip(idX1_temp, idY1_temp)))       
            if (0.05*n_KP_GF) > 5001:
                n1 = 0.05*n_KP_GF  
            else:
                if n_KP_GF > 5001:
                    n1 = 5001
                else:
                    n1 = n_KP_GF                    
            idX1 = []
            idY1 = []
            for key in groups_KP1.keys():                
                random.seed(4)
                list_temp1 = random.sample(groups_KP1[key],len(groups_KP1[key])) 
                if n_KP_GF > 0:
                    percent = len(groups_KP1[key])/n_KP_GF                           
                    n2 = int(percent*n1)              
                    if n2 > 0:
                        for i1 in range(n2):
                            x_temp1, y_temp1 = list_temp1[i1]   
                            idX1.append(x_temp1)
                            idY1.append(y_temp1)           
            rows1 = ['L'+str(index) for index in range(len(arr1_temp))]
            columns1 = [index for index in range(len(idX1))]
            results1 = pd.DataFrame(index=rows1, columns=columns1)
            data1 = [] 
            for idxLayer in range(len(arr1_temp)):
                pixelValues = Parallel(n_jobs=-1)(delayed(getPixelValue)(arr1_temp,idxLayer,idX1[k],idY1[k]) for k in range(len(idX1)))
                data1.append(pixelValues)
            results1 = pd.DataFrame(data1, index=rows1, columns=columns1).T
            results1.insert(loc=0, column='idx', value=idX1)   
            results1.insert(loc=1, column='idy', value=idY1)  
            path1 = os.path.join(subdir1,file1[7:15])
            os.makedirs(path1, exist_ok=True)
            outputdir1 = os.path.join(path1, name+'.xlsx')
            results1.to_excel(outputdir1, encoding='utf-8')

In [ ]:
maindir1 = r'G:\MScThesis\waterQualityMonitoring\Data\MLearning\preparedInputData'
gf_folders = ['withoutGF','withGF']
GF = []
Date = []
S2 = []
Samples = []
for i in tqdm([0,1]):
    gf_folder = gf_folders[i]
    subdir1 = os.path.join(maindir1,gf_folder,'France')    
    for file1 in os.listdir(subdir1):
        if 'tiff' not in file1:
            subdir2 = os.path.join(maindir1,gf_folder,'France',file1)
            for file2 in os.listdir(subdir2):
                excel_file = pd.read_excel(os.path.join(subdir1,file1,file2))
                GF.append(gf_folder)
                Date.append(file1)
                S2.append(file2[12:18])
                Samples.append(excel_file.shape[0])
df = pd.DataFrame({'GF':GF, 'Date':Date, 'S2':S2, 'Samples':Samples})

In [ ]:
outputdir2 = os.path.join(maindir1,'summary.xlsx')
df.to_excel(outputdir2, encoding='utf-8', index=False)